In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# import os 

# dir_path = '/content/drive/MyDrive/checkpoints'
# if os.path.isdir(dir_path):
#     pass
# else:
#     os.mkdir(dir_path)

In [ ]:
# %cp /content/drive/MyDrive/pytorch_model.bin /content/UKYDataScienceComp2023/domain_adaptation_final_body_2/pytorch_model.bin

# %cp /content/drive/MyDrive/online_news_popularity_data.csv /content/UKYDataScienceComp2023/online_news_popularity_data/online_news_popularity_data.csv


In [ ]:
# git and pip install



# !pip install transformers datasets

In [1]:
from datasets import load_dataset

articles = load_dataset('online_news_popularity_data')

Found cached dataset online_news_popularity_data (/home/leepark/.cache/huggingface/datasets/online_news_popularity_data/online_news_popularity_data/1.0.0/f3e03630a13ebe013884d6a83c7ec52cb4eec2c0f6012f710c9dba58aa719fcd)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [3]:
def tokenize(batch):
    # full_text_encoded = tokenizer([title + ' \n' + content for title, content in zip(batch['title'],batch['content'])],
    #                              return_tensors = 'pt', padding = 'max_length', max_length = 512, truncation = True)
    title_encoded = tokenizer(batch['title'], padding = True,  truncation = True)
    title_encoded_renamed = {f"{k}_title":v for k,v in title_encoded.items()}
    content_encoded = tokenizer(batch['content'], padding = 'max_length', max_length = 512, truncation = True)
    content_encoded_renamed = {f"{k}_content":v for k,v in content_encoded.items()}
#     return {k:torch.Tensor(v) for k,v in full_text_encoded.items()}
    title_encoded_renamed.update(content_encoded_renamed)
    return title_encoded_renamed

In [4]:
articles_encoded = articles.map(tokenize, remove_columns = ['title','content','shares'],
            batched = True, batch_size = None)

Loading cached processed dataset at /home/leepark/.cache/huggingface/datasets/online_news_popularity_data/online_news_popularity_data/1.0.0/f3e03630a13ebe013884d6a83c7ec52cb4eec2c0f6012f710c9dba58aa719fcd/cache-97268e3b8769f89b.arrow
Loading cached processed dataset at /home/leepark/.cache/huggingface/datasets/online_news_popularity_data/online_news_popularity_data/1.0.0/f3e03630a13ebe013884d6a83c7ec52cb4eec2c0f6012f710c9dba58aa719fcd/cache-60f76fa598c78ae0.arrow


In [1]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch
import os
from dataclasses import dataclass
from Transformer_Models import ContextDistilBert, ContextDistilBertwithData


@dataclass
class SequenceClassifierOutput:
    loss: torch.FloatTensor = None
    logits: torch.FloatTensor = None
    preds: np.array = None
    labels: torch.FloatTensor = None
    hidden_states: torch.FloatTensor = None


class ContextDistilBertwithDataforClassification(nn.Module):
    def __init__(self, model_dir):
        super().__init__()
        self.body = ContextDistilBertwithData(model_dir)
        self.pre_classifier = nn.Linear(4398, 4398)
        self.classifier = nn.Linear(4398, 2)
        self.dropout = nn.Dropout(.1)
        
    def forward(self, **kwargs):
        labels = kwargs['shares_class']
        distilbert_output = self.body(**kwargs)
        pooled_output = distilbert_output
        pooled_output = self.pre_classifier(pooled_output)
        pooled_output = nn.GELU()(pooled_output)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        softmax = logits.detach().cpu().numpy()
        
        # calculate losses
        loss = None
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits.view(-1,2), labels.view(-1))
            
        result = SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=distilbert_output,
            preds = softmax,
            labels = labels
        )
        

        # return model output object
        return result

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

articles_encoded.set_format('pt')

NameError: name 'articles_encoded' is not defined

In [37]:
from tqdm import tqdm
def train_loop(h_training_dataset, model, optimizer, batch_size, device, verbose = False):
    h_training_dataset = h_training_dataset.shuffle()
    total_rows = h_training_dataset.num_rows
    steps = total_rows // batch_size
    if total_rows % batch_size:
        steps += 1
    if verbose:
        pbar = tqdm(range(steps))
        for batch_idx in pbar:
            start = batch_idx*batch_size; end = batch_idx*batch_size + batch_size
            if end > total_rows:
                result = model(**{k:v.to(device) for k,v in h_training_dataset[start:].items()})
            else:
                result = model(**{k:v.to(device) for k,v in h_training_dataset[start:end].items()})
            optimizer.zero_grad()
            result.loss.backward()
            optimizer.step()
            if batch_idx % 100 == 0:
                loss, current = result.loss.item(), (batch_idx + 1) * steps
                pbar.set_postfix({"mean(loss)":""f"{loss:>7f}  [{current:>5d}/{batch_size:>5d}]"})
    else:
        for batch_idx in range(steps):
            start = batch_idx*batch_size; end = batch_idx*batch_size + batch_size
            if end > total_rows:
                result = model(**{k:v.to(device) for k,v in h_training_dataset[start:].items()})
            else:
                result = model(**{k:v.to(device) for k,v in h_training_dataset[start:end].items()})
            optimizer.zero_grad()
            result.loss.backward()
            optimizer.step()
            if batch_idx % 25 == 0:
                loss, current = result.loss.item(), (batch_idx + 1) * batch_size
                print({"mean(loss)":""f"{loss:>7f}  [{current:>5d}/{total_rows:>5d}]"})


In [38]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def compute_metrics(labels, preds):
    f1 = f1_score(labels, preds)
    acc = accuracy_score(labels, preds)
    recall = recall_score(labels, preds)
    auc = roc_auc_score(labels, preds)
    precision = precision_score(labels, preds)
    return {'accuracy': acc, 'precision': precision, 'recall':recall, 'f1':f1, 'auc':auc}

In [39]:
def test_loop(h_test_dataset, model, batch_size, device, verbose = False):
    h_test_dataset = h_test_dataset.shuffle()
    total_rows = h_test_dataset.num_rows
    steps = total_rows // batch_size
    if total_rows % batch_size:
        steps += 1
    from sklearn.metrics import accuracy_score
    preds = []
    labels = h_test_dataset.with_format('np')['shares_class']
    loss = 0
    with torch.no_grad():
        if verbose:
            pbar = tqdm(range(steps))
            for batch_idx in pbar:
                start = batch_idx*batch_size; end = batch_idx*batch_size + batch_size
                if end > total_rows:
                    result = model(**{k:v.to(device) for k,v in h_test_dataset[start:].items()})
                else:
                    result = model(**{k:v.to(device) for k,v in h_test_dataset[start:end].items()})
                loss += result.loss.item()
                preds.append(np.argmax(result.preds, -1))
            preds = np.concatenate(preds)
            accuracy = accuracy_score(preds.reshape(-1,1), labels.reshape(-1,1))
            loss /= total_rows
            print(f"accuracy : {accuracy}")
            print(f"loss : {loss}")
            return compute_metrics(labels, preds)
        else:
            for batch_idx in range(steps):
                start = batch_idx*batch_size; end = batch_idx*batch_size + batch_size
                if end > total_rows:
                    result = model(**{k:v.to(device) for k,v in h_test_dataset[start:].items()})
                else:
                    result = model(**{k:v.to(device) for k,v in h_test_dataset[start:end].items()})
                loss += result.loss.item()
                preds.append(np.argmax(result.preds, -1))
            preds = np.concatenate(preds)
            accuracy = accuracy_score(preds.reshape(-1,1), labels.reshape(-1,1))
            loss /= (steps + 1)
            print(f"evaluation result : accuracy : {accuracy}, loss : {loss}")
            return compute_metrics(labels, preds)


In [ ]:
import smtplib
from email.mime.text import MIMEText

# Texting function

def send_email(subject, body, sender, recipients, password):
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = sender
    msg['To'] = ', '.join(recipients)
    smtp_server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    smtp_server.login(sender, password)
    smtp_server.sendmail(sender, recipients, msg.as_string())
    smtp_server.quit()

subject = "Test"
body = "This is the body of the text message"
sender = "park.family.bucheon@gmail.com"
recipients = ["8593335321@msg.fi.google.com"]
password = "fizwmiuiwonvuulj"

send_email(subject, body, sender, recipients, password)


In [382]:
model_ckpt = 'domain_adaptation_final_body_2'

model = ContextDistilBertwithDataforClassification(model_ckpt).to(device)


optimizer = torch.optim.AdamW(model.parameters(), lr = 3e-4)

scheduler_1 = torch.optim.lr_scheduler.ConstantLR(optimizer, factor=1, total_iters=1)
scheduler_2 = torch.optim.lr_scheduler.ConstantLR(optimizer, factor=round(1/3, 3), total_iters = 1)
scheduler_3 = torch.optim.lr_scheduler.ConstantLR(optimizer, factor=round(5/30, 3), total_iters = 1)
scheduler_4 = torch.optim.lr_scheduler.ConstantLR(optimizer, factor=.1, total_iters = 1)
lr_scheduler = torch.optim.lr_scheduler.SequentialLR(optimizer, schedulers = [scheduler_1, scheduler_2,
                                                  scheduler_3, scheduler_4],
                                                 milestones=[1,2,3])


for i in range(4):
    train_loop(articles_encoded['train'], model, optimizer, 16, device)
    metrics = test_loop(articles_encoded['validation'].shuffle().select(range(640)), model, 16, device)
    lr_scheduler.step()
    subdir_path = os.path.join(dir_path, f'checkpoint_epoch_{i}')
    os.mkdir(subdir_path)
    file_name = 'model.pth'
    torch.save(model.state_dict(), os.path.join(subdir_path, file_name))
    with open(os.path.join(subdir_path, 'metrics.txt'), 'w') as f:
        f.write(f"summary of ecpoh {i}\n")
        for k,v in metrics.items():
            f.write(f"{k} : {v}\n")
            f.write('end')
        f.close()

{'mean(loss)': '66.784706  [  100/31686]'}
{'mean(loss)': '69.858818  [ 2600/31686]'}
{'mean(loss)': '67.101128  [ 5100/31686]'}
{'mean(loss)': '66.627663  [ 7600/31686]'}
{'mean(loss)': '67.919441  [10100/31686]'}
{'mean(loss)': '65.001122  [12600/31686]'}
{'mean(loss)': '68.053848  [15100/31686]'}
{'mean(loss)': '65.664352  [17600/31686]'}
{'mean(loss)': '69.843575  [20100/31686]'}
{'mean(loss)': '70.365585  [22600/31686]'}
{'mean(loss)': '68.642189  [25100/31686]'}
{'mean(loss)': '68.711418  [27600/31686]'}
{'mean(loss)': '67.622452  [30100/31686]'}
evaluation result : accuracy : 0.5684170663973744, loss : 0.6787905020113095
{'mean(loss)': '68.190559  [  100/31686]'}
{'mean(loss)': '67.504478  [ 2600/31686]'}
{'mean(loss)': '69.684685  [ 5100/31686]'}
{'mean(loss)': '66.440605  [ 7600/31686]'}
{'mean(loss)': '64.930801  [10100/31686]'}
{'mean(loss)': '67.543205  [12600/31686]'}
{'mean(loss)': '66.385651  [15100/31686]'}
{'mean(loss)': '68.732430  [17600/31686]'}
{'mean(loss)': '67.0